In [ ]:
import glob
import os

from nltk.tokenize.punkt import PunktSentenceTokenizer
from nltk import word_tokenize  # Simplfied notation; it's a wrapper for the TreebankWordTokenizer
from nltk.corpus import stopwords

from nltk.help import upenn_tagset
from nltk import pos_tag

from tqdm import tqdm

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
proj = {
    'input_folder': r'data/datasets/bbc/politics',
    'lib': 'nltk',  # 'nltk' or 'spacy', preferred library for tasks like tokenization and pos tagging
}

feat_list = {
    'path',
    'name',
    'text',
    'length',
    'sentences'
    'tokens',
    'pos_tags',
    'count_sentences',
    'count_tokens',
}

feat = {}
for f in feat_list:
    feat[f] = f

In [ ]:
proj['input_folder']

In [ ]:
docs = []
paths = glob.glob(os.path.join(proj['input_folder'], '*'))
for path in tqdm(paths):
    if os.path.isfile(path):
        with open(path, 'r') as file:
            text = file.read()
            docs.append({
                'path': path,
                'name': os.path.basename(path),
                'text': text,
                'length': len(text),
            })

print("document 0: {0} characters".format(docs[0]['length']))

print("Number of files from the dataset:", len(docs))
print(docs[0]['name'])

In [ ]:
sentence_tokenizer = PunktSentenceTokenizer()

for doc in docs:
    doc['sentences'] = sentence_tokenizer.tokenize(doc['text'])
    doc['count_sentences'] = len(doc['sentences'])
    
for doc in docs:
    doc['tokens'] = []
    for s in doc['sentences']:
        doc['tokens'] += word_tokenize(s)
    doc['count_tokens'] = len(doc['tokens'])

## Go thru all the documents, and then take the first 75 tokens and save it

In [ ]:
all_first_tokens = []
for doc in docs:
    first_n_tokens = doc['tokens'][:75]
    all_first_tokens.append(first_n_tokens)

In [ ]:
print(all_first_tokens[1])
print(len(all_first_tokens))

In [ ]:
import pickle
with open("first_tokens.pickle", "wb") as f:
    pickle.dump(all_first_tokens, f)

## Feed tokens to LLM and generate new documents

In [1]:
import pickle
all_first_tokens = pickle.load(open("first_tokens.pickle", "rb"))

In [2]:
model_name = "microsoft/phi-2"


In [3]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
device = "cuda"

model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)
model.to(device)

tokenizer = AutoTokenizer.from_pretrained(model_name)

/home/genliang/python-env/text-mining/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████████████| 2/2 [00:00<00:00,  3.68it/s]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
# Model generation parameters, tweak around max_length and temperature for more creative outputs
# https://huggingface.co/docs/transformers/en/main_classes/text_generation#transformers.GenerationConfig
generation_parameters = {
    "max_length": 1024,
    "temperature": 0.9,
    "top_k": 10,
    "top_p": 0.95,
    "repetition_penalty": 1.2,
    "num_return_sequences": 1,
    "do_sample": True,
    # "eos_token_id": tokenizer.eos_token_id
}

In [7]:
from tqdm import tqdm

In [14]:
no_words = 512 # no of words to generate

for i in tqdm(range(len(all_first_tokens))):
# for i in range(2):
    first_words = ' '.join(all_first_tokens[i])
    # print(first_words)
    prompt = f'''
    Instruct: Generate news article with 1000 words about politics with these starting words "{first_words}" Output:
    '''
    # prompt = f'''
    # Instruct: Complete a news article with 1000 words about politics with these starting words "{first_words}" Output:
    # '''
    model_inputs = tokenizer([prompt], return_tensors="pt").to(device)
    generated_ids = model.generate(**model_inputs, **generation_parameters)
    generated_ids_without_prompt = generated_ids[0][len(model_inputs['input_ids'][0]):].unsqueeze(0)
    output_results = tokenizer.batch_decode(generated_ids_without_prompt, skip_special_tokens=False)[0]
    # print(output_results)
    with open(f"result_{i}.txt", "w", encoding="utf-8") as f:
        f.write(output_results)

 32%|█████████████▏                           | 134/417 [07:06<12:15,  2.60s/it]Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
 37%|███████████████▎                         | 156/417 [08:07<09:08,  2.10s/it]Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
 47%|███████████████████▏                     | 195/417 [10:12<11:16,  3.05s/it]Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
 50%|████████████████████▍                    | 208/417 [10:54<09:39,  2.77s/it]Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
 55%|██████████████████████▍                  | 228/417 [11:45<08:24,  2.67s/it]Setting `pad_token_id` t

In [9]:
output_results

'"""\n\n    text = text_generator.sample(n=512)\n    print (f\'generated: {len(text)}\') <|endoftext|>'

In [ ]:
first_words